# 预处理得到数据集

In [1]:
import pandas as pd

question_list_path = [f'./question_list/question_list{i}.txt' for i in range(1,5)]
feature = []
label = []
num = 0
for question_path in question_list_path :
    with open(question_path,'r') as f :
        content = f.read().split("\n")
    feature += content
    label += [num for i in range(len(content))]
    num += 1

dataset =pd.DataFrame([feature,label]).transpose()
dataset.columns = ['feature','label']
dataset.to_excel('./dataset.xlsx',index=None)

# 处理数据集

In [2]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split

## 读取预训练大模型

In [3]:
from InstructorEmbedding import INSTRUCTOR

# 读取本地模型文件
model = INSTRUCTOR("F:\\工作以及比赛\\大一统框架\\dd\\try2\\t1\\model")

e:\Anaconda\envs\myproject1\lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
function 'cadam32bit_grad_fp32' not found


e:\Anaconda\envs\myproject1\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


max_seq_length  512


## 将语料库中的语句转化为句向量

In [4]:
import torch
# 读取数据
dataset = pd.read_excel('./dataset.xlsx')
# 转化为句向量
dataset = np.concatenate([model.encode(dataset['feature'].to_numpy()),torch.nn.functional.one_hot(torch.tensor(dataset['label'].to_numpy()),num_classes=4)],axis=1)
# 划分训练集和测试集
traindataset,testdataset = train_test_split(dataset,test_size=0.3,shuffle=True)
# 划分特征空间和标签
feature_train,label_train = traindataset[:,0:-4],traindataset[:,traindataset.shape[-1]-4:traindataset.shape[-1]]
feature_test,label_test = testdataset[:,0:-4],testdataset[:,testdataset.shape[-1]-4:testdataset.shape[-1]]
# 转换为torch风格的dataset
traindataset = torch.utils.data.TensorDataset(torch.Tensor(feature_train),torch.Tensor(label_train))
testdataset = torch.utils.data.TensorDataset(torch.Tensor(feature_test),torch.Tensor(label_test))
# 定义批处理器
# batch_size = 10 
# traindataloader = torch.utils.data.DataLoader(traindataset,batch_size=batch_size,drop_last=True)
# testdataloader = torch.utils.data.DataLoader(testdataset,batch_size=batch_size,drop_last=True)

# 准备训练

In [5]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

def score_func(batch_size,hidden_size,drop_prod,num_layers,activate_func,lr,epochs) :
    # 预处理bayes优化传的参数
    batch_size = int(batch_size)
    hidden_size = int(hidden_size)
    epochs = int(epochs)
    num_layers = int(num_layers)
    activate_func = ['relu','sigmoid','tanh'][int(activate_func)]
    
    # 定义批处理器
    traindataloader = torch.utils.data.DataLoader(traindataset,batch_size=batch_size,drop_last=True)
    testdataloader = torch.utils.data.DataLoader(testdataset,batch_size=batch_size,drop_last=True)

    # 定义神经网络结构
    class network(nn.Module) :
        def __init__(self,input_size,hidden_size,drop_prod,num_layers,activate_func) :
            super().__init__()
            # 激活函数可供选择
            if activate_func == 'relu' :
                acf = nn.ReLU()
            elif activate_func == 'sigmoid' :
                acf = nn.Sigmoid()
            elif activate_func == 'tanh' :
                acf = nn.Tanh()
            
            layer_list = [
                nn.Linear(in_features = input_size,out_features = hidden_size,bias = True),
                acf,
                nn.Dropout(p=drop_prod),
            ]

            for i in range(num_layers-1) :
                layer_list.append(nn.Linear(in_features = hidden_size,out_features = hidden_size,bias = True))
                layer_list.append(acf)
                layer_list.append(nn.Dropout(p=drop_prod))
            
            layer_list.append(nn.Linear(in_features = hidden_size,out_features = 4 ,bias = True))
            self.model = nn.Sequential(*layer_list)

        def forward(self,x) :
            return self.model(x)

    # 实例化网络
    input_size = dataset.shape[-1] - 4
    # hidden_size = 256
    # drop_prod = 0.8
    net = network(input_size=input_size,hidden_size=hidden_size,drop_prod=drop_prod,num_layers=num_layers,activate_func=activate_func)

    # 迁移模型
    # net.to("cuda:0")

    # 定义优化器
    # lr = 1e-3
    optimizer =torch.optim.Adam(net.parameters(),lr)

    # 定义损失函数
    loss = nn.CrossEntropyLoss()

    # 初始化参数
    for name,param in net.named_parameters() :
        if "bias" in name :
            nn.init.constant_(param,val=0)
        elif "weight" in name :
            nn.init.normal_(param,mean=0,std=1)

    # epochs = 3000
    for epoch in range(epochs) :
        ls = 0
        # 开启训练模式
        net.train()
        for pi_X,pi_y in traindataloader :
            # 迁移数据
            # pi_X = pi_X.to("cuda:0")
            # pi_y = pi_y.to("cuda:0")
            l = loss(net(pi_X),pi_y)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()
            ls += l
        lt = 0
        a = 0
        t = 0
        with torch.no_grad():
            # 开启评估模式
            net.eval()
            for pi_X_test,pi_y_test in testdataloader :
                # pi_X_test = pi_X_test.to("cuda:0")
                # pi_y_test = pi_y_test.to("cuda:0")
                lt += loss(net(pi_X_test),pi_y_test)
                a += ((net(pi_X_test).argmax(dim=1) == pi_y_test.argmax(dim=1))).sum()
                t += pi_X_test.shape[0]
            ac = a / t
            # print(ls,lt,a)

    return ac , net

In [6]:
batch_size_bayes = (5,100)
hidden_size_bayes = (100,400)
drop_prod_bayes = (0.2,0.8)
num_layers_bayes = (1,5)
activate_func_bayes = (0,2)
lr_bayes = (1e-3,5e-2)
epochs_bayes = (1000,3000)

In [7]:
from bayes_opt import BayesianOptimization

score_func_score = lambda **kwargs : score_func(**kwargs)[0]

param_dict = {
    "batch_size":batch_size_bayes,
    "hidden_size":hidden_size_bayes,
    "drop_prod":drop_prod_bayes,
    "num_layers":num_layers_bayes,
    "activate_func":activate_func_bayes,
    "lr":lr_bayes,
    "epochs":epochs_bayes,
}
opt = BayesianOptimization(score_func_score,param_dict)

In [8]:
opt.maximize(n_iter=15, init_points=3)
print(opt.max)

|   iter    |  target   | activa... | batch_... | drop_prod |  epochs   | hidden... |    lr     | num_la... |
-------------------------------------------------------------------------------------------------------------
| 1         | 0.2128    | 0.07772   | 47.63     | 0.6244    | 2.744e+03 | 161.5     | 0.04823   | 4.549     |
| 2         | 0.2581    | 0.1231    | 31.73     | 0.3555    | 1.575e+03 | 170.7     | 0.03088   | 4.635     |
| 3         | 0.2525    | 1.315     | 33.03     | 0.4891    | 1.603e+03 | 309.2     | 0.009012  | 3.381     |
| 4         | 0.23      | 2.0       | 100.0     | 0.8       | 1e+03     | 100.0     | 0.00973   | 5.0       |
| 5         | 0.2685    | 0.08511   | 27.04     | 0.5955    | 1.57e+03  | 182.8     | 0.00428   | 2.206     |
| 6         | 0.27      | 0.5922    | 25.68     | 0.5915    | 1.568e+03 | 188.7     | 0.04698   | 4.631     |
| 7         | 0.2368    | 0.81      | 19.7      | 0.4831    | 1.502e+03 | 195.8     | 0.04332   | 1.203     |
| 8       

In [9]:
score_func_model = lambda **kwargs : score_func(**kwargs)[-1]

mynet = score_func_model(**opt.max['params'])

In [10]:
mynet.eval()

# mynet(torch.Tensor(feature_test))
mynet

network(
  (model): Sequential(
    (0): Linear(in_features=768, out_features=244, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.32255092815907316, inplace=False)
    (3): Linear(in_features=244, out_features=4, bias=True)
  )
)

In [11]:
sum(mynet(torch.Tensor(feature_test)).argmax(dim=1) == torch.Tensor(label_test).argmax(dim=1)) / label_test.__len__()

tensor(0.5833)

In [12]:
mynet(torch.Tensor(feature_test)).argmax(dim=1)

tensor([1, 2, 1, 3, 1, 3, 1, 2, 0, 0, 3, 0, 2, 0, 2, 3, 1, 3, 0, 0, 3, 2, 0, 0,
        2, 1, 1, 3, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 3, 3, 1, 3, 2,
        2, 1, 3, 1, 0, 3, 0, 3, 3, 0, 2, 3, 2, 0, 3, 2, 3, 0, 1, 2, 2, 3, 3, 3,
        2, 3, 0, 3, 3, 3, 0, 2, 2, 3, 0, 3, 1, 0, 1, 0, 3, 2, 1, 3, 3, 2, 3, 2,
        2, 1, 2, 3, 1, 3, 0, 2, 1, 1, 0, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 3, 1, 0])

In [45]:
import json
# 保存模型参数
torch.save(mynet.state_dict(),"./model_params.pth")

# 保存网络结构参数
with open("./model_struct.json","w",encoding = "utf8") as f :
    f.write(json.dumps(opt.max["params"]))

# 加载网络结构参数
with open("./model_struct.json","r") as f :
    content = f.read()
model_struct = json.loads(content)

# 定义网络结构
class network(nn.Module) :
    def __init__(self,input_size,hidden_size,drop_prod,num_layers,activate_func) :
        super().__init__()
        # 激活函数可供选择
        if activate_func == 'relu' :
            acf = nn.ReLU()
        elif activate_func == 'sigmoid' :
            acf = nn.Sigmoid()
        elif activate_func == 'tanh' :
            acf = nn.Tanh()
        
        layer_list = [
            nn.Linear(in_features = input_size,out_features = hidden_size,bias = True),
            acf,
            nn.Dropout(p=drop_prod),
        ]

        for i in range(num_layers-1) :
            layer_list.append(nn.Linear(in_features = hidden_size,out_features = hidden_size,bias = True))
            layer_list.append(acf)
            layer_list.append(nn.Dropout(p=drop_prod))
        
        layer_list.append(nn.Linear(in_features = hidden_size,out_features = 4 ,bias = True))
        self.model = nn.Sequential(*layer_list)

    def forward(self,x) :
        return self.model(x)

# 定义网络结构参数
hidden_size = model_struct["hidden_size"]
drop_prod = model_struct["drop_prod"]
num_layers = model_struct["num_layers"]
activate_func = model_struct["activate_func"]

# 定义模型
model_ll = network(input_size=768,hidden_size=int(hidden_size),drop_prod=drop_prod,num_layers=int(num_layers),activate_func=["relu","sigmoid","tanh"][int(activate_func)])

# 加载模型参数
model_ll.load_state_dict(torch.load("./model_params.pth"))
model_ll.eval()

network(
  (model): Sequential(
    (0): Linear(in_features=768, out_features=244, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.32255092815907316, inplace=False)
    (3): Linear(in_features=244, out_features=4, bias=True)
  )
)

In [44]:
sum(model_ll(torch.Tensor(feature_test)).argmax(dim=1) == torch.Tensor(label_test).argmax(dim=1)) / label_test.__len__()

tensor(0.5833)